In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [2]:
from langchain.document_loaders import WebBaseLoader

#add website data
URL = ["https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/",
       "https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/"]

#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)

In [4]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import os
#from getpass import getpass

# get your free access token from HuggingFace and paste it here
#HF_token = getpass()
#os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token
from dotenv import load_dotenv
load_dotenv()
 
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")


#embeddings = HuggingFaceInferenceAPIEmbeddings(api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunking,embeddings)

C:\Users\338566\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "what is recurrent neural network?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)

[Document(id='5532fe83-d2c4-49af-b44f-3aca17ea3ffa', metadata={'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US'}, page_content='Recurrent Neural Networks Explanation'), Document(id='efd8ee79-e1ea-49a9-966c-e095fa3de0bf', metadata={'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programmi

C:\Users\338566\AppData\Local\Temp\ipykernel_13000\4027306976.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_rel = retriever.get_relevant_documents(query)


In [7]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [8]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
                       model_kwargs={"temperature":0.5,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])

C:\Users\338566\AppData\Local\Temp\ipykernel_13000\3221105185.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa(prompt)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Recurrent Neural Networks (RNNs) are ANNs designed to identify patterns in sequences of data, such as time-series or language data. Unlike traditional feedforward networks, RNNs have a unique structure where the output of each step is used as the input

Recurrent Neural Networks Explanation

Bidirectional Recurrent Neural Network

Question: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
what is recurrent neural network?
</s>
<|assistant|>

Helpful Answer:

Recurrent Neural Networks (RNNs) are a type of Artificial Neural Network (ANN) that are designed to identify patterns in sequences of data, such as time-series or language data. Unlike traditional feedforward networks, RNNs ha

In [9]:
vectorstore.save_local("huggingface")

In [10]:
new_db = FAISS.load_local("huggingface", embeddings,allow_dangerous_deserialization=True)
new_db.similarity_search(query)

[Document(id='5532fe83-d2c4-49af-b44f-3aca17ea3ffa', metadata={'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US'}, page_content='Recurrent Neural Networks Explanation'),
 Document(id='7e4320c4-992a-41d8-bf47-23b8da60582d', metadata={'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks', 'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programm